In [24]:
import os
import glob
import re
import json
import numpy as np
import pandas as pd
from nilearn import image, signal
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import resample_to_img
import argparse
import logging
from itertools import combinations
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any, Union
import warnings
import sys

In [25]:
from nilearn.datasets import fetch_coords_power_2011

In [29]:
def fetch_nilearn_atlas(atlas_name: str, atlas_params: Dict[str, Any], logger: logging.Logger) -> Tuple[Any, Dict[int, str]]:
    """Fetch a built-in Nilearn atlas and return the image and network labels."""
    try:
        if atlas_name not in NILEARN_ATLASES:
            raise ValueError(f"Unknown Nilearn atlas: {atlas_name}. Available: {list(NILEARN_ATLASES.keys())}")
        
        atlas_info = NILEARN_ATLASES[atlas_name]
        fetch_func = atlas_info['function']
        
        if fetch_func is None:
            raise ValueError(f"Atlas function for {atlas_name} is not available in this Nilearn version")
        
        # Merge default parameters with user parameters
        params = atlas_info['default_params'].copy()
        params.update(atlas_params)
        
        logger.info(f"Fetching {atlas_name} atlas with parameters: {params}")
        
        # Fetch the atlas
        try:
            atlas_data = fetch_func(**params)
        except OSError as e:
            error_msg = str(e)
            if "File exists" in error_msg or "nilearn_data" in error_msg:
                logger.warning("Detected Nilearn data directory issue, attempting to fix...")
                try:
                    fix_nilearn_data_directory()
                    logger.info("Retrying atlas fetch after fixing data directory...")
                    atlas_data = fetch_func(**params)
                except Exception as retry_e:
                    logger.error(f"Failed to fetch {atlas_name} atlas after fixing data directory: {str(retry_e)}")
                    logger.error("This might be due to:")
                    logger.error("1. Network connectivity issues")
                    logger.error("2. Insufficient disk space")
                    logger.error("3. Nilearn version compatibility")
                    logger.error("4. Persistent data directory issues")
                    logger.error("Try running with --fix-nilearn-data option")
                    raise
            else:
                logger.error(f"Failed to fetch {atlas_name} atlas: {error_msg}")
                logger.error("This might be due to:")
                logger.error("1. Network connectivity issues")
                logger.error("2. Insufficient disk space")
                logger.error("3. Nilearn version compatibility")
                raise
        except Exception as e:
            logger.error(f"Failed to fetch {atlas_name} atlas: {str(e)}")
            logger.error("This might be due to:")
            logger.error("1. Network connectivity issues")
            logger.error("2. Insufficient disk space")
            logger.error("3. Nilearn version compatibility")
            raise
        
        # Load the atlas image
        try:
            atlas_img = image.load_img(atlas_data['maps'])
        except Exception as e:
            logger.error(f"Failed to load atlas image: {str(e)}")
            raise
        
        # Generate network labels
        network_labels = {}
        if atlas_info['network_based']:
            # For network-based atlases, generate network labels
            if atlas_name == 'schaefer_2018' and 'network_names' in atlas_info:
                # Use predefined network names for Schaefer 2018
                yeo_networks = params.get('yeo_networks', 7)
                if yeo_networks in atlas_info['network_names']:
                    network_names = atlas_info['network_names'][yeo_networks]
                    # Get the number of ROIs per network
                    n_rois = params.get('n_rois', 400)
                    rois_per_network = n_rois // yeo_networks
                    
                    roi_idx = 1
                    for network_idx, network_name in enumerate(network_names):
                        for _ in range(rois_per_network):
                            network_labels[roi_idx] = network_name
                            roi_idx += 1
                    
                    # Handle any remaining ROIs
                    while roi_idx <= n_rois:
                        network_labels[roi_idx] = network_names[roi_idx % yeo_networks]
                        roi_idx += 1
                    
                    logger.info(f"Generated Schaefer 2018 network labels: {yeo_networks} networks, {n_rois} ROIs")
                else:
                    logger.warning(f"Unknown number of networks for Schaefer 2018: {yeo_networks}")
                    # Fall back to generic approach
                    if 'labels' in atlas_data:
                        labels = atlas_data['labels']
                        for i, label in enumerate(labels):
                            if isinstance(label, str):
                                # Extract network name from label
                                if '_' in label:
                                    network_name = label.rsplit('_', 1)[0]
                                else:
                                    network_name = label
                                network_labels[i + 1] = network_name
                            else:
                                network_labels[i + 1] = f"Network_{i+1}"
                    else:
                        # Generate generic network labels
                        n_rois = len(np.unique(atlas_img.get_fdata())) - 1  # Exclude background (0)
                        for i in range(n_rois):
                            network_labels[i + 1] = f"Network_{i+1}"
            elif 'labels' in atlas_data:
                labels = atlas_data['labels']
                for i, label in enumerate(labels):
                    if isinstance(label, str):
                        # Extract network name from label
                        if '_' in label:
                            network_name = label.rsplit('_', 1)[0]
                        else:
                            network_name = label
                        network_labels[i + 1] = network_name
                    else:
                        network_labels[i + 1] = f"Network_{i+1}"
            else:
                # Generate generic network labels
                n_rois = len(np.unique(atlas_img.get_fdata())) - 1  # Exclude background (0)
                for i in range(n_rois):
                    network_labels[i + 1] = f"Network_{i+1}"
        else:
            # For non-network atlases, generate anatomical labels
            n_rois = len(np.unique(atlas_img.get_fdata())) - 1  # Exclude background (0)
            for i in range(n_rois):
                network_labels[i + 1] = f"Region_{i+1}"
        
        logger.info(f"Successfully fetched {atlas_name} atlas: {len(network_labels)} ROIs, shape={atlas_img.shape}")
        logger.info(f"Generated {len(set(network_labels.values()))} unique network labels")
        
        return atlas_img, network_labels
        
    except Exception as e:
        logger.error(f"Failed to fetch {atlas_name} atlas: {str(e)}")
        raise

In [30]:
# Set up the test
print("Setting up Power 2011 atlas test...")

# Create a simple logger for testing
import logging
logger = logging.getLogger('FC_Analysis')
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

print("✅ Logger set up successfully")
print("Note: Power 2011 atlas is handled specially - it doesn't use a standard Nilearn fetch function")
print("Instead, it loads from local files or generates from coordinates")
print("Let's test the Power 2011 coordinate loading directly in the next cell...")

Setting up Power 2011 atlas test...
✅ Logger set up successfully
Note: Power 2011 atlas is handled specially - it doesn't use a standard Nilearn fetch function
Instead, it loads from local files or generates from coordinates
Let's test the Power 2011 coordinate loading directly in the next cell...


In [ ]:
# Test the ACTUAL Power 2011 atlas loading logic from NW_1st.py
print("Testing the REAL Power 2011 atlas loading logic from NW_1st.py...")

def test_power_2011_atlas_loading(logger):
    """Test the actual Power 2011 atlas loading logic from NW_1st.py lines 1697-1814"""
    try:
        # This is the actual logic from NW_1st.py lines 1704-1710
        from nilearn.datasets import fetch_coords_power_2011
        power = fetch_coords_power_2011()
        required_fields = ['roi', 'x', 'y', 'z']
        
        # This is the fix we made - use dtype.names instead of columns
        if not all(field in power.rois.dtype.names for field in required_fields):
            logger.error(f"Power.rois missing required fields {required_fields}: {power.rois.dtype.names}")
            raise ValueError("Invalid Power 2011 atlas data structure")
        
        # Test coordinate extraction (lines 1711-1712)
        coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T
        logger.info(f"Power 2011 coordinates shape: {coords.shape} (expected: (264, 3))")
        
        # Test the network labels loading logic (lines 1754-1806)
        n_rois = len(power.rois)  # 264 ROIs
        
        # This is the other fix we made - no .values needed (line 1762)
        if not np.array_equal(power.rois['roi'], np.arange(1, n_rois + 1)):
            logger.error("Power.rois['roi'] does not contain 1–264 in order")
            raise ValueError("Invalid ROI numbers in power.rois")
        
        # Load network labels using the SAME logic as NW_1st.py (lines 1766-1776)
        # Use cluster Power 2011 network labels if available
        cluster_power_labels = '/scratch/xxqian/roi/power264/power264NodeNames.txt'
        if os.path.exists(cluster_power_labels):
            network_labels_path = cluster_power_labels
            logger.info(f"Using cluster Power 2011 network labels: {network_labels_path}")
        else:
            # Fallback to local path (for testing)
            network_labels_path = '/Users/xiaoqianxiao/tool/parcellation/power264/power264NodeNames.txt'
            logger.info(f"Using local Power 2011 network labels: {network_labels_path}")
        
        if not os.path.exists(network_labels_path):
            logger.error(f"Network labels file not found: {network_labels_path}")
            raise FileNotFoundError(f"Missing {network_labels_path}")
        
        try:
            with open(network_labels_path, 'r') as f:
                network_labels_list = [line.strip() for line in f if line.strip()]
            if len(network_labels_list) != n_rois:
                logger.error(f"Network labels file has {len(network_labels_list)} entries, expected {n_rois}")
                raise ValueError(f"Invalid number of network labels in {network_labels_path}")
            
            network_labels = {}
            for i, (label, roi_num) in enumerate(zip(network_labels_list, power.rois['roi'])):
                parts = label.rsplit('_', 1)  # This is the actual parsing logic (line 1787)
                if len(parts) == 2:
                    network_name, txt_roi_num = parts
                    try:
                        txt_roi_num = int(txt_roi_num)
                    except ValueError:
                        logger.error(f"Invalid ROI number in {network_labels_path} at line {i+1}: {txt_roi_num}")
                        raise ValueError(f"Invalid ROI number: {txt_roi_num}")
                    if txt_roi_num != roi_num:
                        logger.error(f"ROI number mismatch in {network_labels_path} at line {i+1}: {txt_roi_num} != {roi_num}")
                        raise ValueError(f"ROI number mismatch: {txt_roi_num} != {roi_num}")
                    network_labels[i + 1] = network_name
                else:
                    logger.error(f"Invalid label format in {network_labels_path} at line {i+1}: {label}")
                    raise ValueError(f"Invalid label format: {label}")
            
            logger.info(f"Loaded {len(network_labels)} network labels from {network_labels_path}")
            
        except Exception as e:
            logger.error(f"Failed to load network labels from {network_labels_path}: {str(e)}")
            raise
        
        logger.info(f"Loaded {len(network_labels)} network labels")
        logger.info(f"Loaded Power 2011 atlas: 264 ROIs, {len(set(network_labels.values()))} networks")
        
        return power, network_labels, coords
        
    except Exception as e:
        logger.error(f"Failed to load Power 2011 atlas: {str(e)}")
        raise

# Test the actual function
try:
    power, network_labels, coords = test_power_2011_atlas_loading(logger)
    print("✅ Power 2011 atlas loaded successfully using REAL logic from NW_1st.py!")
    print(f"✅ ROIs: {len(power.rois)}")
    print(f"✅ Coordinates shape: {coords.shape}")
    print(f"✅ Network labels: {len(network_labels)}")
    print(f"✅ Unique networks: {len(set(network_labels.values()))}")
    print(f"✅ Networks: {list(set(network_labels.values()))}")
    print("\n🎉 All fixes are working correctly!")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


2025-09-02 11:37:41,814 - FC_Analysis - INFO - Power 2011 coordinates shape: (264, 3) (expected: (264, 3))
2025-09-02 11:37:41,814 - FC_Analysis - INFO - Power 2011 coordinates shape: (264, 3) (expected: (264, 3))
2025-09-02 11:37:41,814 - FC_Analysis - INFO - Power 2011 coordinates shape: (264, 3) (expected: (264, 3))
2025-09-02 11:37:41,814 - FC_Analysis - INFO - Power 2011 coordinates shape: (264, 3) (expected: (264, 3))
2025-09-02 11:37:41,814 - FC_Analysis - INFO - Power 2011 coordinates shape: (264, 3) (expected: (264, 3))
2025-09-02 11:37:41,815 - FC_Analysis - INFO - Loading Power 2011 network labels from: /Users/xiaoqianxiao/tool/parcellation/power264/power264NodeNames.txt
2025-09-02 11:37:41,815 - FC_Analysis - INFO - Loading Power 2011 network labels from: /Users/xiaoqianxiao/tool/parcellation/power264/power264NodeNames.txt
2025-09-02 11:37:41,815 - FC_Analysis - INFO - Loading Power 2011 network labels from: /Users/xiaoqianxiao/tool/parcellation/power264/power264NodeNames.t

Testing the REAL Power 2011 atlas loading logic from NW_1st.py...
✅ Power 2011 atlas loaded successfully using REAL logic from NW_1st.py!
✅ ROIs: 264
✅ Coordinates shape: (264, 3)
✅ Network labels: 264
✅ Unique networks: 14
✅ Networks: ['Cingulo-opercular_Task_Control', 'Default_mode', 'Fronto-parietal_Task_Control', 'Memory_retrieval', 'Sensory_Somatomotor_Mouth', 'Dorsal_attention', 'Salience', 'Sensory_Somatomotor_Hand', 'Uncertain', 'Ventral_attention', 'Subcortical', 'Visual', 'Cerebellar', 'Auditory']

🎉 All fixes are working correctly!


In [36]:
power

{'rois': rec.array([(  1, -25, -98, -12), (  2,  27, -97, -13),
            (  3,  24,  32, -18), (  4, -56, -45, -24),
            (  5,   8,  41, -24), (  6, -21, -22, -20),
            (  7,  17, -28, -17), (  8, -37, -29, -26),
            (  9,  65, -24, -19), ( 10,  52, -34, -27),
            ( 11,  55, -31, -17), ( 12,  34,  38, -12),
            ( 13,  -7, -52,  61), ( 14, -14, -18,  40),
            ( 15,   0, -15,  47), ( 16,  10,  -2,  45),
            ( 17,  -7, -21,  65), ( 18,  -7, -33,  72),
            ( 19,  13, -33,  75), ( 20, -54, -23,  43),
            ( 21,  29, -17,  71), ( 22,  10, -46,  73),
            ( 23, -23, -30,  72), ( 24, -40, -19,  54),
            ( 25,  29, -39,  59), ( 26,  50, -20,  42),
            ( 27, -38, -27,  69), ( 28,  20, -29,  60),
            ( 29,  44,  -8,  57), ( 30, -29, -43,  61),
            ( 31,  10, -17,  74), ( 32,  22, -42,  69),
            ( 33, -45, -32,  47), ( 34, -21, -31,  61),
            ( 35, -13, -17,  75), ( 36, 

In [37]:
network_labels

{1: 'Uncertain',
 2: 'Uncertain',
 3: 'Uncertain',
 4: 'Uncertain',
 5: 'Uncertain',
 6: 'Uncertain',
 7: 'Uncertain',
 8: 'Uncertain',
 9: 'Uncertain',
 10: 'Uncertain',
 11: 'Uncertain',
 12: 'Uncertain',
 13: 'Sensory_Somatomotor_Hand',
 14: 'Sensory_Somatomotor_Hand',
 15: 'Sensory_Somatomotor_Hand',
 16: 'Sensory_Somatomotor_Hand',
 17: 'Sensory_Somatomotor_Hand',
 18: 'Sensory_Somatomotor_Hand',
 19: 'Sensory_Somatomotor_Hand',
 20: 'Sensory_Somatomotor_Hand',
 21: 'Sensory_Somatomotor_Hand',
 22: 'Sensory_Somatomotor_Hand',
 23: 'Sensory_Somatomotor_Hand',
 24: 'Sensory_Somatomotor_Hand',
 25: 'Sensory_Somatomotor_Hand',
 26: 'Sensory_Somatomotor_Hand',
 27: 'Sensory_Somatomotor_Hand',
 28: 'Sensory_Somatomotor_Hand',
 29: 'Sensory_Somatomotor_Hand',
 30: 'Sensory_Somatomotor_Hand',
 31: 'Sensory_Somatomotor_Hand',
 32: 'Sensory_Somatomotor_Hand',
 33: 'Sensory_Somatomotor_Hand',
 34: 'Sensory_Somatomotor_Hand',
 35: 'Sensory_Somatomotor_Hand',
 36: 'Sensory_Somatomotor_Hand',
 

In [38]:
coords

array([[-25, -98, -12],
       [ 27, -97, -13],
       [ 24,  32, -18],
       [-56, -45, -24],
       [  8,  41, -24],
       [-21, -22, -20],
       [ 17, -28, -17],
       [-37, -29, -26],
       [ 65, -24, -19],
       [ 52, -34, -27],
       [ 55, -31, -17],
       [ 34,  38, -12],
       [ -7, -52,  61],
       [-14, -18,  40],
       [  0, -15,  47],
       [ 10,  -2,  45],
       [ -7, -21,  65],
       [ -7, -33,  72],
       [ 13, -33,  75],
       [-54, -23,  43],
       [ 29, -17,  71],
       [ 10, -46,  73],
       [-23, -30,  72],
       [-40, -19,  54],
       [ 29, -39,  59],
       [ 50, -20,  42],
       [-38, -27,  69],
       [ 20, -29,  60],
       [ 44,  -8,  57],
       [-29, -43,  61],
       [ 10, -17,  74],
       [ 22, -42,  69],
       [-45, -32,  47],
       [-21, -31,  61],
       [-13, -17,  75],
       [ 42, -20,  55],
       [-38, -15,  69],
       [-16, -46,  73],
       [  2, -28,  60],
       [  3, -17,  58],
       [ 38, -17,  45],
       [-49, -11